# Data Cleanup and Preprocessing

Before using the CIC-IDS 2017 Dataset, the data has to be preprocessed and cleaned. The raw files are 7 csv files containing the recorded network traffic for 5 working days with benign traffic and various attacks (Brute Force Attack, Heart Bleed Attack, Botnet, Dos Attack, DDos Attack, Web Attack (SQL Injection, XSS, Brute Force), Infiltration Attack)

In [1]:
import numpy as np
import pandas as pd
import os

## 1. Exploring one file from the dataset

To understand the dataset, one file is loaded and analyzed before processing all of the files

In [2]:
dataset_path = r"CIC-IDS-2017\CSVs\GeneratedLabelledFlows\TrafficLabelling"
file_path = os.path.join(dataset_path, "Monday-WorkingHours.pcap_ISCX.csv")
df = pd.read_csv(file_path)
# Remove space in column names using strip() function
df.rename(columns=lambda x: x.strip(), inplace=True)
# Remove columns unnecessary for machine learning
df = df.drop(columns=['Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Timestamp'])
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 529918 entries, 0 to 529917
Data columns (total 80 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Destination Port             529918 non-null  int64  
 1   Protocol                     529918 non-null  int64  
 2   Flow Duration                529918 non-null  int64  
 3   Total Fwd Packets            529918 non-null  int64  
 4   Total Backward Packets       529918 non-null  int64  
 5   Total Length of Fwd Packets  529918 non-null  float64
 6   Total Length of Bwd Packets  529918 non-null  float64
 7   Fwd Packet Length Max        529918 non-null  float64
 8   Fwd Packet Length Min        529918 non-null  float64
 9   Fwd Packet Length Mean       529918 non-null  float64
 10  Fwd Packet Length Std        529918 non-null  float64
 11  Bwd Packet Length Max        529918 non-null  float64
 12  Bwd Packet Length Min        529918 non-null  float64
 13 

Convert label type to category

In [3]:
convert_dict = {'Label': 'category'}
df = df.astype(convert_dict)

Check for infinity and null values in one of the columns

In [4]:
print(f"Infinity values of flow_byts_s: {df[df['Flow Bytes/s'] == np.inf]['Destination Port'].count()}")
print(f"Null values of flow_byts_s: {df[df['Flow Bytes/s'].isnull()]['Destination Port'].count()}")

Infinity values of flow_byts_s: 373
Null values of flow_byts_s: 64


## 2. Cleanup all Files

The "Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv" contains a problematic character that has to be replaced before processing all files and empty lines at the end of the file.

In [5]:
problematic_file = "Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv"
problematic_file_path = os.path.join(dataset_path, problematic_file)
with open(problematic_file_path, 'rb') as file:
    content = file.read()
# Replace the problematic character (0x96) with a hyphen (-)
content_fixed = content.replace(b'\x96', b'-')
# Split the content into lines and remove lines that contain only commas
lines = content_fixed.decode('utf-8').split('\n')
cleaned_lines = [line for line in lines if not all(char == ',' for char in line.strip())]
with open(problematic_file_path, 'wb') as file:
    file.write('\n'.join(cleaned_lines).encode('utf-8'))

To cleanup the rest of the files:
1. Trim column names of whitespaces and convert them to lowercase
2. Drop the columns that are unnecessary for machine learning

In [6]:
import re
files = {
    "Monday-WorkingHours.pcap_ISCX.csv": "Monday.csv",
    "Tuesday-WorkingHours.pcap_ISCX.csv": "Tuesday.csv",
    "Wednesday-workingHours.pcap_ISCX.csv": "Wednesday.csv",
    "Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv": "Thursday-Morning-WebAttacks.csv",
    "Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv": "Thursday-Afternoon-Infiltration.csv",
    "Friday-WorkingHours-Morning.pcap_ISCX.csv": "Friday-Morning.csv",
    "Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv": "Friday-Afternoon-Portscan.csv",
    "Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv": "Friday-Afternoon-DDos.csv"}
column_name_regex = re.compile(r"\W", re.IGNORECASE)
processed_dir = "processed"
processed_path = os.path.join(dataset_path, processed_dir)
def trim_column_names(df):
    return [column_name_regex.sub('_', c.lower()) for c in df.columns]
if not os.path.exists(processed_path):
    os.mkdir(processed_path) 
for file_in, file_out in files.items():
    file_path = os.path.join(dataset_path, file_in)
    output_path = os.path.join(processed_path, file_out)
    df = pd.read_csv(file_path)
    df.rename(columns=lambda x: x.strip(), inplace=True)
    df = df.drop(columns=['Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Timestamp'])
    df.columns = trim_column_names(df)
    df.to_csv(output_path, index=False)
    print("Labels for file:", file_in)
    print(df['label'].value_counts())

Labels for file: Monday-WorkingHours.pcap_ISCX.csv
label
BENIGN    529918
Name: count, dtype: int64
Labels for file: Tuesday-WorkingHours.pcap_ISCX.csv
label
BENIGN         432074
FTP-Patator      7938
SSH-Patator      5897
Name: count, dtype: int64
Labels for file: Wednesday-workingHours.pcap_ISCX.csv
label
BENIGN              440031
DoS Hulk            231073
DoS GoldenEye        10293
DoS slowloris         5796
DoS Slowhttptest      5499
Heartbleed              11
Name: count, dtype: int64
Labels for file: Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
label
BENIGN                        168186
Web Attack - Brute Force        1507
Web Attack - XSS                 652
Web Attack - Sql Injection        21
Name: count, dtype: int64
Labels for file: Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
label
BENIGN          288566
Infiltration        36
Name: count, dtype: int64
Labels for file: Friday-WorkingHours-Morning.pcap_ISCX.csv
label
BENIGN    189067
Bot         1

## 3. Data Preparation
All of the processed datasets are grouped into one Pandas dataframe to analyze the content. The data is then saved into one single csv file.

In [7]:
import glob
csv_files = glob.glob(os.path.join(processed_path, '*.csv'))
df = pd.concat((pd.read_csv(f) for f in csv_files))

In [8]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2830743 entries, 0 to 225744
Data columns (total 80 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   destination_port             int64  
 1   protocol                     int64  
 2   flow_duration                int64  
 3   total_fwd_packets            int64  
 4   total_backward_packets       int64  
 5   total_length_of_fwd_packets  float64
 6   total_length_of_bwd_packets  float64
 7   fwd_packet_length_max        float64
 8   fwd_packet_length_min        float64
 9   fwd_packet_length_mean       float64
 10  fwd_packet_length_std        float64
 11  bwd_packet_length_max        float64
 12  bwd_packet_length_min        float64
 13  bwd_packet_length_mean       float64
 14  bwd_packet_length_std        float64
 15  flow_bytes_s                 float64
 16  flow_packets_s               float64
 17  flow_iat_mean                float64
 18  flow_iat_std                 float64
 19  flow_i

#### Creating labels for the attacks

In [9]:
import re
df['is_attack'] = df.label.apply(lambda x: 0 if x == "BENIGN" else 1)
convert_dict = {'label': 'category'}
df = df.astype(convert_dict)
# Having attack types as integers can be helpful for some machine learning algorithms
df['label_code'] = df['label'].cat.codes
attacks = df["label"].value_counts().index.tolist()
for attack in attacks:
    if attack != "BENIGN":
        attack = attack.lower().replace('-', '')
        l = "is_" + re.sub(r'\s+', '_', attack)
        df[l] = df.label.apply(lambda x: 1 if x == attack else 0)
df.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
Index: 2830743 entries, 0 to 225744
Data columns (total 96 columns):
 #   Column                       Dtype   
---  ------                       -----   
 0   destination_port             int64   
 1   protocol                     int64   
 2   flow_duration                int64   
 3   total_fwd_packets            int64   
 4   total_backward_packets       int64   
 5   total_length_of_fwd_packets  float64 
 6   total_length_of_bwd_packets  float64 
 7   fwd_packet_length_max        float64 
 8   fwd_packet_length_min        float64 
 9   fwd_packet_length_mean       float64 
 10  fwd_packet_length_std        float64 
 11  bwd_packet_length_max        float64 
 12  bwd_packet_length_min        float64 
 13  bwd_packet_length_mean       float64 
 14  bwd_packet_length_std        float64 
 15  flow_bytes_s                 float64 
 16  flow_packets_s               float64 
 17  flow_iat_mean                float64 
 18  flow_iat_std                

#### Saving the grouped dataset to a single file

In [10]:
output_path = os.path.join(processed_path, "ids2017_processed.csv")
df.to_csv(output_path, index = False)